In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta


import xarray as xr
import matplotlib.pyplot as plt
import sys
# sys.path.append(r'C:\Users\ruro\OneDrive - Boskalis\Documents\python\OSSI')
from KNMI_readers import read_knmi_uurgeg
from scipy.signal import welch

import puv 


In [2]:
def solo_data_reader(dataFile, sf):
    '''
    Function to read solo datafile.
    Returns a dataframe with a time column and pressure column in Pascal
    '''
    p = []
    datt = []
    with open(dataFile) as myfile:
        for index, line in enumerate(myfile):
            if index >= 1:
                lin = line.split(',')
                datt.append(lin[0])
                p.append(float(lin[1]))
    p = np.array(p) * 1e4  # dBar to Pa

    t = pd.date_range(datt[0], periods=len(datt), freq='{}S'.format(1 / sf))

    dfp = pd.DataFrame(data={'p': p}, index=t)

    dfp.index.name = 't'
    return dfp

In [3]:
# # input parameters
# # RBRSolo^3
sf = 16  #[hz] sampling frequency
experimentFolder = r'O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\S3P3 RBR6'                  # path where the data is sitting # Rubens Laptop
# experimentFolder = r'U:\'                  # path where the data is sitting # TUD U-Drive


knmiFile = r"C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\uurgeg_330_2021-2030.txt"  # path to the KNMI data
instrumentName = instrument = 'RBR_06_data1'                                                                              # designated name of the instrument
namedatafile = 'RBR6 - 208682_20241227_1744_data.txt'                                                          # name of the datafile
dataFile =  os.path.join(experimentFolder, namedatafile)                                                 # path + name datafile
serial_number = '202438'                                                                                 # unique serial number of the instrument
xRD = 72398.448                                                                                          # x position of placement in field
yRD =  452130.253                                                                                        # y position of placement in field
zRD_bedlevel =zb = -1.229         # m NAP
zRD_sensor = zi =  -0.704         # m NAP


# Do the reading from file and cast in xarray dataset
dfp = solo_data_reader(dataFile, sf)
ds = dfp.to_xarray()
ds.p.attrs = {'long_name': 'pressure', 'units': 'Pa'}

# Add global attribute metadata
ds.attrs = {
    'Conventions': 'CF-1.6',
    'name': '{}'.format(instrumentName),
    'instrument': '{}'.format(instrumentName),
    'instrument type': 'Ruskin RBR Solo',
    'instrument serial number': '{}'.format(serial_number),
    'epsg': 28992,
    'x': xRD,
    'y': yRD,
    'time zone': 'UTC+2',
    'coordinate type': 'XYZ',
    'summary': 'Hybrid-Dune experiment',
    'contact person': 'Ruben Rosman',
    'emailadres': 'r.g.c.rosman@student.tudelft.nl',
    'construction datetime': datetime.now().strftime("%d-%b-%Y (%H:%M:%S)"),
    'version': 'v1',
    'version comments': 'constructed with xarray'}

# Save to netcdf
ncOutDir = os.path.join(experimentFolder, 'raw_netcdf')
if not os.path.isdir(ncOutDir):
    os.mkdir(ncOutDir)
#ds.to_netcdf(os.path.join(ncOutDir, instrumentName + '.nc'))

C:\Users\dpoppema\AppData\Local\Temp\ipykernel_37296\2651882304.py:16: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  t = pd.date_range(datt[0], periods=len(datt), freq='{}S'.format(1 / sf))


In [ ]:
# frequency resolution in fourier space
### delta_f = p_blocks/D_timeframe ###
p_blocks = 20
D_length = 1200
fresolution = p_blocks / D_length
print(f'fresolution is: {fresolution}')
nperseg = D_length * sf / p_blocks - 0.5 #dim should be len(ds.f); whelch has (nperseg/2 +1)

# -----------------------------------------------------------------------------------
rho = 10277 #kg/m3, for seawater at 9C, avg temp at HKZ measurement station
g = 9.81

dataFile = os.path.join(experimentFolder,'raw_netcdf',instrumentName + '.nc')
print(dataFile)
ds0 = xr.open_dataset(dataFile)
instr = ds0.instrument

# correct for the air pressure fluctuations and drift in the instrument
# first we load the data and add it to the dataset
dfp = read_knmi_uurgeg(knmiFile, stationNumber=330)
dt = ((ds0.t[1] - ds0.t[0]) / np.timedelta64(1, 's')).values # target frequency
pAir = dfp['P'].to_xarray().sel(t=slice(ds0.t.min(), ds0.t.max())).resample({'t': '{}S'.format(dt)}).interpolate('linear')
ds0['pAir'] = pAir.sel(t=slice(ds0.t.min(), ds0.t.max()))

# correct the pressure signal with pAir
ds0['pc'] = ds0['p'] -ds0['pAir']
# remove all values below 0 Pa
# ds0['pc'] = ds0['pc'].where(ds0['pc'] > 0)

ds0['pc'].attrs = {'units': 'Pa', 'long_name': 'pressure', 'comments': 'corrected for air pressure'}

# -----------------------------------------------------------------------------
# reshape to one row per burst in data array
pt = ds0.pc.values
nSamples = len(pt)
dt = ds0.isel(t=1).t - ds0.isel(t=0).t
sf = np.timedelta64(1, 's') / dt.values

D_length = '1200S'

burstDuration = pd.Timedelta(D_length)  # Burst duration (1200 seconds = 20 minutes)
burstLength = int(burstDuration / dt)
nBursts = int(np.floor(nSamples / burstLength))

pt = pt[:nBursts * burstLength]
t = ds0.t[::burstLength]
t = t[:nBursts]
N = (ds0.t.values[:burstLength] - ds0.t.values[0]) / np.timedelta64(1, 's')
# pdb.set_trace()

# --------------------------------------------------------------------------
# cast into a 2D array
ds = xr.Dataset(data_vars={},
                coords={'t': t, 'N': N})
# copy all data over into this new structure
ds['p'] = (('t', 'N'), pt.reshape((nBursts, burstLength)))
ds['zi'] = zi
ds['zb'] = zb
ds['sf'] = sf



# Remove water height below 30 cm
ds['p'] = ds['p'].where(ds['p'] > 0.3 * (rho * g))

# Remove bursts where the standard deviation is too low, indicating the instrument fell dry
ds['p'] = ds['p'].where(ds.p.std(dim='N') > 70)

# --------------------------------------------------------------------------
# pdb.set_trace()
ds['p'].attrs = {'units': 'Pa +NAP', 'long_name': 'pressure', 'comments': 'corrected for air pressure'}
ds['zi'].attrs = {'units': 'm+NAP', 'long_name': 'z instrument'}  # instrument height
ds['zb'].attrs = {'units': 'm+NAP', 'long_name': 'z bed'}  # bed level  
ds['sf'].attrs = {'units': 'Hz', 'long_name': 'sampling frequency'}
ds.attrs = ds0.attrs
ds.attrs['summary'] = 'Hybrid-Dune campaign, pressure corrected for air pressure and cast in bursts of 20 minutes'
ds['name'] = instr
if not os.path.isdir(os.path.join(experimentFolder,'QC')):
    os.mkdir(os.path.join(experimentFolder,'QC'))
ncFilePath = os.path.join(experimentFolder, 'QC', instr + '.nc')
#ds.to_netcdf(ncFilePath)

fresolution is: 0.016666666666666666
O:\HybridDune experiment\data RBR, OSSI\copy RBR Udrive series1\S3P3 RBR6\raw_netcdf\RBR_06_data1.nc
RBR_06_data1


In [ ]:
dfp.P.plot()
start_time = pd.Timestamp("2024-12-13T16:00:00")
end_time = pd.Timestamp("2024-12-27T13:00:00")
plt.xlim(start_time, end_time)
pt
# plt.xlim(ds0.t.min(), ds0.t.max())

In [ ]:
# ds0.p.plot()
ds0.pc.plot()
plt.grid()


In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T16:00:00")
end_time = pd.Timestamp("2024-12-23T13:00:00")
plt.xlim(start_time, end_time)

In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T20:00:00")
end_time = pd.Timestamp("2024-12-18T13:00:00")
plt.xlim(start_time, end_time)

In [ ]:
plt.figure(figsize=(16,2))
ds0.pc.plot()
plt.grid()

start_time = pd.Timestamp("2024-12-17T20:00:00")
end_time = pd.Timestamp("2024-12-17T20:05:00")
plt.xlim(start_time, end_time)
plt.ylim(-100, 20000)

In [ ]:
instrFile = os.path.join(experimentFolder,'QC', instrument +'.nc')
ncOutFile = os.path.join(experimentFolder,'tailored', instrument +'.nc')


# %% load the raw data from netcdf
ds0 = xr.open_dataset(instrFile)

#let's remove the bursts where there are only nans
ds0 = ds0.dropna(dim='t')

# make a new dataset that has an extra dimension to accomodate for the frequency axis
ds = xr.Dataset(data_vars={},
                coords={'t': ds0.t.values,
                        'N': ds0.N.values,
                        'f': np.arange(0, ds0.sf.values / 2, fresolution)})
ds['f'].attrs = {'units': 'Hz'}
ds.attrs = ds0.attrs

# put all variables in this new dataset
for key in ds0.data_vars:
    ds[key] = ds0[key]

# extract sampling frequency as explicit variable
sf = ds.f.values

def remove_outliers(data, threshold=3):
    mean = np.mean(data)
    std_dev = np.std(data)
    mask = np.abs(data - mean) < threshold * std_dev
    # filtered_data = np.where(mask, data, mean + threshold * std_dev * np.sign(data - mean))  # Replace outliers with closest value within threshold
    filtered_data = np.where(mask, data, np.nan) # Replace outliers with nan
    return filtered_data

# Create a lambda function to pass the threshold parameter
ufunc = lambda x: remove_outliers(x, threshold=3)

# Apply the outlier removal function to the pressure data
ds['p'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

# Interpolate NaN values in the pressure signal
ds['p'] = ds['p'].interpolate_na(dim='N', method='linear')


ds.p.isel(t=111).plot()

In [ ]:
# Print the initial number of NaN values
initial_nan_count = np.sum(np.isnan(ds['p'].values))
print("Initial NaN count:", initial_nan_count)

# Interpolating NaN values using linear interpolation
ds['p'] = ds['p'].interpolate_na(dim='N', method='linear')

# Handling any remaining NaN values
ds['p'] = ds['p'].ffill(dim='N').bfill(dim='N')

# Print the number of NaN values after interpolation
remaining_nan_count = np.sum(np.isnan(ds['p'].values))
print("Remaining NaN count:", remaining_nan_count)

In [ ]:

### pressure filtering ####
ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.004, fmax=1.5)

ds['p'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)


# Compute water depth
ds['h'] = (ds['p'] / (rho * g) + (ds['zi'] - ds['zb'])).mean(dim='N')  # Relative to bed level
ds['h'].attrs = {'long_name': 'mean water level', 'units': '[m] above bed level'}

In [ ]:
ds

In [ ]:
# Several wave statistics computations, only based on pressure, for full bandpass
onoff=False
Threshold= 4


ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1025,
    removeNoise=onoff,
    detrend=True)

fx, ds['zs'] = xr.apply_ufunc(ufunc, 
                              ds['p'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)

# ds['zs'] = xr.apply_ufunc(ufunc, 
#                           ds['zs'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# # Interpolate NaN values in the pressure signal
# ds['zs'] = ds['zs'].interpolate_na(dim='N', method='linear')

# Drop values that are below bed level. 
# ds['zs'] = ds['zs'].where(ds['zs'] >= 0.0, drop=True)

ds['zs'].attrs = {'units': 'm', 'long_name': 'surface elevation'}

ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann')

ds['frequencies'], ds['psd'] = xr.apply_ufunc(ufunc,
                                                ds['zs'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.004 , fmax=5)

ds['Hm0'], ds['Tp'], ds['Tm01'], ds['Tm02'], ds['Tmm10'], ds['Tps'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)



In [ ]:
## low frequencies: IG waves ##
## now first filter bandpass, then compute wave_params

ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.004, fmax=0.05)

ds['p_low'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1025,
    removeNoise=onoff,
    detrend=True)

fx, ds['zs_low'] = xr.apply_ufunc(ufunc, 
                              ds['p_low'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

# ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)

# ds['zs_low'] = xr.apply_ufunc(ufunc, 
#                           ds['zs_low'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# ds['zs_low'] = ds['zs_low'].interpolate_na(dim='N', method='linear')

# ds['zs_low'] = ds['zs_low'].where(ds['zs_low'] >= 0.0, drop=True)

ds['zs_low'].attrs = {'units': 'm', 'long_name': 'surface elevation low freq.'}


ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann')

ds['frequencies_low'], ds['psd_low'] = xr.apply_ufunc(ufunc,
                                                ds['zs_low'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.004 , fmax=0.05)

ds['Hm0_low'], ds['Tp_low'], ds['Tm01_low'], ds['Tm02_low'], ds['Tmm10_low'], ds['Tps_low'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd_low'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)




In [ ]:
## High frequencies: wind-waves ##
## now first filter bandpass, then compute wave_params

ufunc = lambda x: puv.band_pass_filter2(ds.sf.values, x, fmin=0.05, fmax=5)

ds['p_high'] = xr.apply_ufunc(ufunc, 
                          ds['p'],
                          input_core_dims=[['N']],
                          output_core_dims=[['N']],
                          vectorize=True)

ufunc = lambda x, h: puv.attenuate_signal(
    'pressure', 
    ds.sf.values, x, h, 
    ds.zi.values,
    ds.zb.values,
    rho=1025,
    removeNoise=onoff,
    detrend=True)

t, ds['zs_high'] = xr.apply_ufunc(ufunc, 
                              ds['p_high'], ds['h'],
                              input_core_dims=[['N'], []],
                              output_core_dims=[['f'], ['N']],
                              vectorize=True)

ufunc = lambda zs: remove_outliers(zs, threshold=Threshold)

# ds['zs_high'] = xr.apply_ufunc(ufunc, 
#                           ds['zs_high'],
#                           input_core_dims=[['N']],
#                           output_core_dims=[['N']],
#                           vectorize=True)

# ds['zs_high'] = ds['zs_high'].interpolate_na(dim='N', method='linear')

# ds['zs_high'] = ds['zs_high'].where(ds['zs_high'] >= 0.0, drop=True)

ds['zs_high'].attrs = {'units': 'm', 'long_name': 'surface elevation high freq.'}



ufunc = lambda p: welch(p, fs=ds.sf.values, nperseg=nperseg, detrend='constant', window='hann')

ds['frequencies_high'], ds['psd_high'] = xr.apply_ufunc(ufunc,
                                                ds['zs_high'],
                                                input_core_dims=[['N']],
                                                output_core_dims=[['f'], ['f']],
                                                vectorize=True)

ufunc = lambda psd: puv.compute_wave_params(ds.f.values, psd, fmin=0.05 , fmax=5)

ds['Hm0_high'], ds['Tp_high'], ds['Tm01_high'], ds['Tm02_high'], ds['Tmm10_high'], ds['Tps_high'] = xr.apply_ufunc(ufunc,
                                                                          ds['psd_high'],
                                                                          input_core_dims=[['f']],
                                                                          output_core_dims=[[], [], [], [], [], []],
                                                                          vectorize=True)


In [ ]:
## skewness of waves ##
ufunc = lambda p: puv.compute_SkAs(sf,p,fpfac =None, fbounds = None)

ds['Sk'], ds['As'], ds['sig'] =  xr.apply_ufunc(ufunc,
                                                ds['p'], 
                                                input_core_dims=[['N']],
                                                output_core_dims=[[], [], []],
                                                vectorize=True)


In [ ]:
# Flatten all variables with dimension 'N'
# for var in ds.data_vars:
#     if 'N' in ds[var].dims:
#         ds[var] = ds[var].stack(t_N=('t', 'N'))

# %% write to file
# we strip all information on burst scale from the dataset to reduce size (and this info is already present in the raw_netcdf version of the data)
# dsTailored = ds.drop_dims('N')
dsTailored = ds


if not os.path.isdir(os.path.join(experimentFolder,'tailored')):
    os.mkdir(os.path.join(experimentFolder,'tailored'))
ncFilePath = os.path.join(experimentFolder, 'tailored', ds0.instrument + '.nc')
dsTailored.to_netcdf(ncFilePath)

In [ ]:
ds

In [ ]:
# Create a continuous time array
t_continuous = np.array([t + np.timedelta64(int(n), 's') for t in ds.t.values for n in ds.N.values])

# Flatten the burst structure for plotting
zs_flat = ds.zs.values.flatten()
zs_low_flat = ds.zs_low.values.flatten()
zs_high_flat = ds.zs_high.values.flatten()


In [ ]:
# Plotting
plt.figure(figsize=(15, 8))

plt.plot(t_continuous, zs_flat, label='zs', alpha=0.7, linewidth=0.9)
plt.plot(t_continuous, zs_high_flat, label='zs_high', alpha=0.7, linewidth=0.9)
plt.plot(t_continuous, zs_low_flat, label='zs_low', alpha=0.7, linewidth=0.9)

ds.h.plot(label='Waterlevel')



# give values below y=0 a different color
# plt.fill_between(t_continuous, zs_flat, 0, where=zs_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
# plt.fill_between(t_continuous, zs_low_flat, 0, where=zs_low_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
# plt.fill_between(t_continuous, zs_high_flat, 0, where=zs_high_flat < 0, color='grey', alpha=1, zorder=4, linewidth=1.5)
plt.axhline(y=0, color='k', linestyle='--', label='Bed level')

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-18T12:00:00")
end_time = pd.Timestamp("2024-12-21T00:00:00")
# plt.xlim(start_time, end_time)

plt.axhline((ds.zi-ds.zb).values, color='k', linestyle='--', label='Instrument height above bed')

plt.xlabel('Time')
plt.ylabel('Surface Elevation (m)')
plt.title('Surface Elevation vs Time RBR06 2024-12-16 to 2024-12-27')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
# plot hmo 
plt.figure(figsize=(15, 8))
ds.Hm0.plot(label='Hm0')
ds.Hm0_low.plot(label=f'Hm0_low (mean={ds.Hm0_low.mean().values:.3g} m)')

ds.Hm0_high.plot(label='Hm0_high')
plt.legend()

plt.xlabel('Time')
plt.ylabel('Hm0 (m)')
plt.title('Hm0 vs Time')
plt.grid(True)

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-18T12:00:00")
end_time = pd.Timestamp("2024-12-21T00:00:00")
# plt.xlim(start_time, end_time)

plt.show()


In [ ]:
# plot hs and h
plt.figure(figsize=(16, 3))
ds.h.plot(label='Waterlevel')
# (ds.h + ds.zb).plot(label='Waterlevel + zb (NAP)')

# Interpolate between points
ds.Hm0.plot(label=f'Hm0 (mean={ds.Hm0.mean().values:.3g} m)')
ds.Hm0_low.plot(label=f'Hm0_low (mean={ds.Hm0_low.mean().values:.3g} m)')

# ds.Hm0_high.plot(label='Hm0_high')
plt.legend()

plt.xlabel('Time')
plt.ylabel('(m)')
plt.title('Hm0 vs Time')
plt.grid(True)

# Set x-axis limits using specific date stamps
start_time = pd.Timestamp("2024-12-21T16:00:00")
end_time = pd.Timestamp("2024-12-23T13:00:00")
# plt.xlim(start_time, end_time)

plt.show()

In [ ]:
t = 111
end =120

plt.figure(figsize=(16, 2))
(ds.zs.isel(t=t)-ds.zs.isel(t=t).mean()).plot(label = f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
# (ds.zs.isel(t=t)-ds.h.isel(t=t).mean()).plot(label = f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0,end)
plt.ylabel(f'\u03B7 [m]')
plt.legend(loc='upper right')

plt.figure(figsize=(16, 2))
ds.zs.isel(t=t).plot(color='grey', label= 'total elevation')
ds.zs_low.isel(t=t).plot(color='green', label = f'infragravity waves; Hm0: {ds.Hm0_low.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0,end)
plt.legend(loc='upper right')

plt.figure(figsize=(16, 2))
ds.zs.isel(t=t).plot(color='grey', label= 'total elevation')
ds.zs_high.isel(t=t).plot(color='red', label = f'wind waves; Hm0: {ds.Hm0_high.isel(t=t).values:.3g}m')

plt.grid()
plt.xlim(0,end)
plt.legend(loc='upper right');

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to remove outliers
# def remove_outliers(data, threshold=3):
#     mean = np.mean(data)
#     std_dev = np.std(data)
#     filtered_data = [x if abs(x - mean) < threshold * std_dev else mean for x in data]
#     return np.array(filtered_data)

def remove_outliers(data, threshold=3):
    mean = np.mean(data)
    std_dev = np.std(data)
    mask = np.abs(data - mean) < threshold * std_dev
    # filtered_data = np.where(mask, data, mean + threshold * std_dev * np.sign(data - mean))  # Replace outliers with closest value within threshold
    filtered_data = np.where(mask, data, np.nan) # Replace outliers with nan
    return filtered_data


data = ds.zs.isel(t=t).values  # Extract the data values for the selected time index t

# Remove outliers from the data
cleaned_data = remove_outliers(data)

# Interpolate NaN values in the pressure signal
# cleaned_data = cleaned_data.interpolate_na(method='linear')

# Calculate the mean of the cleaned data
mean_cleaned_data = np.mean(cleaned_data)

# Plotting the cleaned data
plt.figure(figsize=(16, 2))
plt.plot(ds.N, cleaned_data - mean_cleaned_data, label=f'Water elevation; Hm0: {ds.Hm0.isel(t=t).values:.3g}m')
plt.grid()
plt.xlim(0, 200)
plt.ylabel(f'\u03B7 [m]')
plt.legend(loc='upper right')
plt.show()

print(len(cleaned_data)/16)

In [ ]:
from scipy.stats import chi2
t=111
freq = ds.frequencies.isel(t=t)
psd = ds.psd.isel(t=t)


# Calculate confidence intervals
nBlocks = len(ds.zs.isel(t=t)) // nperseg  # Estimate the number of blocks used by Welch's method
edf = round(nBlocks * 2)     # Degrees of freedom (approximately 2 per segment)
alpha = 0.1                  # 90% confidence level

confLow = edf / chi2.ppf(1 - alpha / 2, edf)  # Lower confidence limit
confUpper = edf / chi2.ppf(alpha / 2, edf)    # Upper confidence limit

# Confidence interval bounds for PSD
psd_lower = psd * confLow
psd_upper = psd * confUpper



### background plot 
freq_bg, psd_bg = welch(ds.zs.isel(t=t).values, fs=16, nperseg=19200, detrend='constant', window='hann')
nBlocks_bg = len(ds.zs.isel(t=t).values) // 19200  # Estimate the number of blocks used by Welch's method
edf_bg = round(nBlocks * 2)     # Degrees of freedom (approximately 2 per segment)
confLow_bg = edf / chi2.ppf(1 - alpha / 2, edf)  # Lower confidence limit
confUpper_bg = edf / chi2.ppf(alpha / 2, edf)    # Upper confidence limit
psd_lower_bg = psd_bg * confLow_bg
psd_upper_bg = psd_bg * confUpper_bg
# plt.plot(freq_bg, psd_upper_bg, color='gray', linestyle= '--')
plt.fill_between(freq_bg, psd_lower_bg, psd_upper_bg, color='gray', alpha=0.3, label='Raw spectrum', linestyle='--')

### plotting
plt.fill_between(freq, psd_lower, psd_upper, color='black', alpha=0.3, label='90% Confidence Interval')
plt.semilogy(freq, psd, label='Power Density Spectrum')

plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (m²/Hz)')
plt.title(f'Power Spectral Density using Welch\'s Method for t ={t}')
plt.grid()
plt.yscale('linear')
# plt.xscale('log')
plt.xscale('linear')
plt.xlim(0,0.6)
plt.legend()
plt.show()

In [ ]:
def find_time_index(ds, specific_time):
    """
    Function to find the index 't' for a specific time in the dataset.
    
    Parameters:
    ds (xarray.Dataset): The dataset containing the time dimension.
    specific_time (str or datetime): The specific time to find the index for.
    
    Returns:
    int: The index corresponding to the specific time.
    """
    specific_time = np.datetime64(specific_time)
    time_index = np.where(ds.t.values == specific_time)[0]
    
    if len(time_index) == 0:
        raise ValueError(f"Time {specific_time} not found in the dataset.")
    
    return time_index[0]

# Example usage
specific_time = "2024-12-20T12:00:00"
t_index = find_time_index(ds, specific_time)
print(f"The index for the specific time {specific_time} is {t_index}.")

In [ ]:
ds